In [1]:
import numpy as np
import pandas as pd


Baseline count: 
if Woman or Children
and Wealthy
    Then Survive
   
Else: 
    Die

In [2]:
def baseLine(row):
    if row.Pclass == 1:
        survived = 1
    elif row.Sex == 'female':
        survived = 1
    elif row.Age <= 12:
        survived = 1
    else:
        survived = 0
    return str(row.PassengerId) + " " + str(survived)

def baselinePredict(X):
    predict = X.apply(baseLine, axis = 1)
    return  pd.DataFrame(predict.str.split(' ',1).tolist(),
                         columns = ['PassengerId','Predict']).apply(pd.to_numeric)

In [3]:
def score(test):
    
   
    #baseline Score
    predict = baselinePredict(test)
    
    merged = pd.merge(test,predict)
    answers = pd.read_csv('gender_submission.csv')
    final = pd.merge(merged, answers, on = 'PassengerId')
    print((final.Survived == final.Predict * 1).sum()/ merged.shape[0])
    

In [23]:
#first GO Titanic Clean
#Could make Class and easier
class cleaner(object):
    def __init__(self):
        self.meanPrice = []
        
    def ageSet(self,row):
        if row.Age <= 12:
            age = 0
        elif row.Age <= 18:
            age = 1
        elif row.Age <= 50:
            age = 2
        elif row.Age > 50:
            age = 3
        else:
            age = 4
        return age

    def aloneSet(self,row):
        #TODO Create status for if have parent or child
        #If traveling alone then alone = 1

        alone = 0
        if int(row.SibSp) == 0 and int(row.Parch) == 0:
            alone = 1
        return alone

    def ticketPriceSet(self,row):
        index = row.Pclass
        if row.Fare > row.meanTicketPrice:
            ticketPrice = 1
        else:
            ticketPrice = 0
        return ticketPrice
        
    def titanicClean(self,df):
        #made id an index
        df = df.set_index('PassengerId')

        df = df.drop([])

        # replace age with child young adult elder
        # if age is unknown Elder
        df['AgeClass'] = df.apply(self.ageSet, axis = 1)

        #Create alone
        df['Alone'] = df.apply(self.aloneSet, axis = 1)
                

        #find mean of fare
        for i in np.unique(df['Pclass']):
            self.meanPrice.append(df.Fare[df['Pclass'] == int(i)].mean(axis = 0))
        self.meanPrice = [0] + self.meanPrice

        #create column of array out of mean array
        #need to do this b/c apply function is weird
        tempTicketMean = []
        for index, row in df.iterrows():
            tempTicketMean.append(self.meanPrice[row.Pclass])    
        
        #Create ticket Price Column
        df['meanTicketPrice'] = tempTicketMean
        df['ticketPrice'] = df.apply(self.ticketPriceSet, axis = 1)

        df.Embarked = df.Embarked.fillna(value = 'S')
        #Change female to int
        df['Sex'] = df['Sex'].map({'female': 1, 'male': 0})
        
        
        df['Embarked'] = df['Embarked'].map({'Q': 1, 'S': 0, 'C':2})
        #drop uneeded columns
        df = df.drop(['Name','Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'meanTicketPrice'],axis = 1)
        df = pd.get_dummies(data = df, columns=['Pclass', 'AgeClass', 'Embarked'])
        
        return df

In [24]:
clean = cleaner()
df = pd.read_csv('test.csv')
print(df.columns)
df = clean.titanicClean(df)
df

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


,Sex,Alone,ticketPrice,Pclass_1,Pclass_2,Pclass_3,AgeClass_0,AgeClass_1,AgeClass_2,AgeClass_3,AgeClass_4,Embarked_0,Embarked_1,Embarked_2
PassengerId,,,,,,,,,,,,,,
892,0,1,0,0,0,1,0,0,1,0,0,0,1,0
893,1,0,0,0,0,1,0,0,1,0,0,1,0,0
894,0,1,0,0,1,0,0,0,0,1,0,0,1,0
895,0,1,0,0,0,1,0,0,1,0,0,1,0,0
896,1,0,0,0,0,1,0,0,1,0,0,1,0,0
897,0,1,0,0,0,1,0,1,0,0,0,1,0,0
898,1,1,0,0,0,1,0,0,1,0,0,0,1,0
899,0,0,1,0,1,0,0,0,1,0,0,1,0,0
900,1,1,0,0,0,1,0,1,0,0,0,0,0,1
